In [ ]:
# given ge, finds next ge (with new gene) in the next adj

# e.g., given adj set: {a:h, b:t} ... {b:h, d:t} & ge = b:t, 
# finds {b:h, d:t}, returns d:t 
function find_next_adj(ge::GeneEnd, adj_set::Set{Adjacency}, adj_list::Vector{Adjacency}, processed_adjs::Set{Adjacency}) 
    for adj in adj_set 
        if adj in processed_adjs
            continue 
        end 

        if adj.left.gene == ge.gene
            push!(adj_list, adj)
            push!(processed_adjs, adj)
            ge = other_adjacency_end(adj.left, adj)
            return ge 
        elseif adj.right.gene == ge.gene
            push!(adj_list, adj)
            push!(processed_adjs, adj)
            ge = other_adjacency_end(adj.right, adj)
            return ge 
        end 
    end 
    return other_adjacency_end(ge, adj)
end 

# O(N^2)
# prints adj set as adj list (in order) 
# note that order within the adj don't matter 
function convert_adj_set_as_adj_list(og_adj_set::Set{Adjacency})
    telos = Vector{Adjacency}() 
    adj_set = copy(og_adj_set)
    adj_list = Vector{Adjacency}() 

    # handle linear chroms 
    for adj in adj_set
        if adj.left.gene == Telomere() || adj.right.gene == Telomere() 
            push!(telos, adj)
        end 
    end 

    processed_adjs = Set{Adjacency}() 
    for adj in telos 
        if adj in processed_adjs
            continue 
        end 
        ge = non_telo_end(adj) 
        push!(adj_list, adj)
        push!(processed_adjs, adj)
        while ge.gene != Telomere()  
            ge = find_next_adj(ge, adj_set, adj_list, processed_adjs, adj)
        end 
    end 
    
    # handle circular chroms 
    for adj in adj_set  # only circular left 
        if adj in processed_adjs
            continue 
        end 
        push!(processed_adjs, adj)
        push!(adj_list, adj)
        first_gene = adj.left.gene 
        ge = GeneEnd(adj.right.gene, !adj.right.head)  # ge that we're looking for
        first = true       
        
        while ge.gene != first_gene || first == true  
            ge = find_next_adj(ge, adj_set, adj_list, processed_adjs, adj)

            first = false 
        end   
    end 
    return adj_list
end 